In [2]:
import pandas as pd
import sqlite3

from pathlib import Path

In [3]:
data = Path.cwd().parent / 'data'
con18 = sqlite3.connect(data / 'Nolan_Conaways_2018_data.db')
con25 = sqlite3.connect(data / 'Pitchfork_Album_Reviews_2025_03_02.db')

# Checking for missing authors

* I manually made a list of missing authors, since they weren't that many (<60). 
* For some reason (or various reasons), some author names changed over time. 
* I actually went one by one, trying to see what happened on every missmatch between 2018 and 2025 data.
  - Sometimes author names were just erased from the whole website (like Loren Ludwig, or Luke Buckman, Alan Smithee, etc.)
  - Sometimes they really changed their name "brad nelson" to "ivy nelson"
  - Sometimes minor name corrections were done like "linhardt" -> "lindhardt"


In [5]:
deleted_authors = [
'alan smithee', 'kim shannon', 'kristin sage rockermann', 'kyle reiter', 'kevin adickes',
'leonard pierce', 'liz colville', 'loren ludwig', 'luke buckman', 'mark martelli', 
'martin clark', 'matt kallman', 'matt stephens', 'matt wellins', 'meg zamula', 'mia lily clarke', 
'michael bernstein', 'michael idov', 'michael wartenbe', 'mike bernstein', 'mike orme', 
'natalie moore', 'nathan humpal', 'nathan rooney', 'neil lieberman', 'neil robertson', 
'nicholas b. sylvester', 'noah sutcliffe', 'oliver lache', 'p.j. gallagher', 
'pete nicholson', 'philip shelley', 'philip welsh', 'rachel khong', 'robbie mackey', 
'roque strew', 'ryan kearney', 'sam chennault', 'sara sherr', 'sarah zupko', 
'scott hreha', 'shan fowler', 'spencer owen', 'stephen haag', 'stephen may', 
'stephen trouss', 'steven byrd', 'stosh "piz" piznarski', 'taylor m. clark', 
'will bryant', 'william morris', 'yancey strickler', 'yancey strickler', 
'zach baron', 'malcolm seymour iii', 'zach hammerman', 'zach vowell']

corrections_to_authors_18 = {
    'alex linhardt': 'alex lindhart',
    'alexander lloyd linhardt': 'alex lindhart',
    'alex linhardt': 'alex lindhart',
    'abigail garnett': 'abby garnett',
    "andy o' connor": "andy o'connor",
    'ben scheim': 'benjamin scheim',
    'brad nelson': 'ivy nelson',
    'cory byrom': 'cory d. byrom',
    'dr. andy beta' : 'andy beta',
    'edwin stats houghton': 'edwin “stats” houghton',
    'edwin "stats" houghton': 'edwin “stats” houghton',
    'grayson currin': 'grayson haver currin',
    'jeremy larson': 'jeremy d. larson',
    'jes skolnik': 'jj skolnik',
    'jillian mapes' : 'jill mapes',
    'josh love': 'joshua love',
    'katherine st asaph': 'katherine st. asaph',
    'mark richard-san': 'mark richardson',
    'matthew grosinger': 'matt grosinger',
    'nina mashurova': 'nm mashurov',
    'pj gallagher': 'p.j. gallagher',
    'richard m. juzwiak': 'rich juzwiak',
    'rohan samarth': 'ro samarth',
    'saby reyes kulkarni': 'saby reyes-kulkarni',
    'savy reyes-kulkarni': 'saby reyes-kulkarni',
    's. murray': 'sean murray',
    'sean fennessy': 'sean fennessey',
    'seth colter-walls': 'seth colter walls',
    'stephen deusner': 'stephen m. deusner',
    'stephen m. duesner': 'stephen m. deusner',
    'stephenm. deusner': 'stephen m. deusner',
    'the pitchfork staff' : 'pitchfork',
    'kim fing shannon' : 'kim shannon',
    'matthew stephens' : 'matt stephens',
    'matthew wellins' : 'matt wellins',}


In [6]:
authors_18 = (pd
    .read_sql("SELECT DISTINCT author FROM reviews", con18)
    .assign(
        author = lambda adf: adf
            .author.str.replace('  ','')
            .str.strip()
            .replace(corrections_to_authors_18)
            .str.replace('with help from ','')
            .str.replace(', and', ''),
        erased = lambda adf: adf.author.isin(deleted_authors))
    .drop_duplicates(keep='first', ignore_index=False))

In [7]:
authors_25 = (pd
    .read_sql("SELECT author_id, author FROM authors", con25, index_col='author_id')
    .assign(author = lambda adf: adf
        .author
        .str.replace('  ',' ')
        .str.strip()))

#### Author Matches

I inspected this table using data Wrangler until every single missmatch was analysed.  
The only missmatches left are for reviews with multiple authors. But all those authors still existed in 2025, so no additional work was necessary.  

In [8]:
author_matches = (authors_25
    .reset_index()
    .assign(author_25_lower = lambda adf: adf.author.str.lower())
    .merge(
        authors_18
            .rename(columns={'author':'author_18_lower'})
            .reset_index(names='author_id_18'),
        left_on = 'author_25_lower',
        right_on = 'author_18_lower',
        how = 'outer' ))

author_matches

,author_id,author,author_25_lower,author_id_18,author_18_lower,erased
0,5926045917cea934e4daf5f3,Aaron Leitko,aaron leitko,116.0,aaron leitko,False
1,592604aec31f3f3472b1d545,Abby Garnett,abby garnett,157.0,abby garnett,False
2,5c86b71e077d417a7480ed26,Abby Jones,abby jones,NaN,NaN,NaN
3,592604afc31f3f3472b1d546,Abigail Covington,abigail covington,108.0,abigail covington,False
4,64b16a5f88ee50644a41718d,Adam Dlugacz,adam dlugacz,402.0,adam dlugacz,False
...,...,...,...,...,...,...
737,592604c7c31f3f3472b1d59a,Zachary Lipez,zachary lipez,NaN,NaN,NaN
738,62bb103e5b6a7bdd3ea30c7c,Zara Hedderman,zara hedderman,NaN,NaN,NaN
739,61113e277818868ad0dc1a20,Zhenzhen Yu,zhenzhen yu,NaN,NaN,NaN
740,592604c7c31f3f3472b1d59c,Zoe Camp,zoe camp,1.0,zoe camp,False


# Bringing history back to life

The 2025 pitchfork album reviews dataset has many authors marked as "Pitchfork" (authors that were there on 2018 but not on 2025).  
The task is to take all those deleted authors, give them a unique ID and associate their id to the 2025 reviews, based on the 2018 reviews.  
I will use urls to match records from both years. 


In [9]:
reviews_18 = (pd
    .read_sql("SELECT reviewid, author, url FROM reviews", con18, index_col='reviewid')
    .assign(
        author = lambda adf: adf
            .author.str.replace('  ','')
            .str.strip()
            .replace(corrections_to_authors_18)
            .str.replace('with help from ','')
            .str.replace(', and', ''),)
    .query('author.isin(@deleted_authors)'))
reviews_18

,author,url
reviewid,,
14715,liz colville,http://pitchfork.com/reviews/albums/14715-ghos...
14595,liz colville,http://pitchfork.com/reviews/albums/14595-past...
14675,liz colville,http://pitchfork.com/reviews/albums/14675-forget/
14607,liz colville,http://pitchfork.com/reviews/albums/14607-alib...
14547,liz colville,http://pitchfork.com/reviews/albums/14547-catc...
...,...,...
7339,taylor m. clark,http://pitchfork.com/reviews/albums/7339-axxes...
8677,neil lieberman,http://pitchfork.com/reviews/albums/8677-summe...
7051,shan fowler,http://pitchfork.com/reviews/albums/7051-the-s...


In [10]:
query = """
SELECT r.review_id, u.url 
FROM reviews r 
JOIN urls u ON r.url_id = u.url_id
"""
reviews_25 = (pd
    .read_sql(query, con25)
    .assign(url = lambda adf: adf.url.str.replace('https', 'http'))
    .query('url.isin(@reviews_18.url)'))
reviews_25

,review_id,url
119,5929e2265e6ef959693250f6,http://pitchfork.com/reviews/albums/6622-ampli...
140,5929e239d7a71d1ea569a421,http://pitchfork.com/reviews/albums/4917-the-d...
143,5929e238c0084474cd0c4d38,http://pitchfork.com/reviews/albums/4703-le-ti...
151,5929e23d5e6ef9596932510b,http://pitchfork.com/reviews/albums/1523-bbc-s...
178,5929e250d7a71d1ea569a432,http://pitchfork.com/reviews/albums/7303-songs...
...,...,...
24289,5929e40f5e6ef959693252cf,http://pitchfork.com/reviews/albums/12568-we-a...
24292,5929e41113d197565213d769,http://pitchfork.com/reviews/albums/12565-fun-...
24309,5929e415eb335119a49ef1b2,http://pitchfork.com/reviews/albums/12552-jewels/
24769,5929e094d7a71d1ea569a266,http://pitchfork.com/reviews/albums/13995-lights/


In [11]:
original_authors = (reviews_25
    .merge(reviews_18, on='url')
    .rename(columns={'author':'author_lower'})
    .set_index('review_id'))
original_authors

,url,author_lower
review_id,,
5929e2265e6ef959693250f6,http://pitchfork.com/reviews/albums/6622-ampli...,taylor m. clark
5929e239d7a71d1ea569a421,http://pitchfork.com/reviews/albums/4917-the-d...,neil lieberman
5929e238c0084474cd0c4d38,http://pitchfork.com/reviews/albums/4703-le-ti...,kristin sage rockermann
5929e23d5e6ef9596932510b,http://pitchfork.com/reviews/albums/1523-bbc-s...,kristin sage rockermann
5929e250d7a71d1ea569a432,http://pitchfork.com/reviews/albums/7303-songs...,taylor m. clark
...,...,...
5929e40f5e6ef959693252cf,http://pitchfork.com/reviews/albums/12568-we-a...,mike orme
5929e41113d197565213d769,http://pitchfork.com/reviews/albums/12565-fun-...,roque strew
5929e415eb335119a49ef1b2,http://pitchfork.com/reviews/albums/12552-jewels/,roque strew


#### Give old authors a new ID

In [12]:
import hashlib

def get_id_hash(name):
    # Function to generate a 32-character ID with prefix "restored"
    hash_part = hashlib.md5(name.encode()).hexdigest()[:24]
    return f"restored{hash_part}"

authors_old = pd.DataFrame({a: get_id_hash(a) for a in deleted_authors}.items(), columns=["author", "author_id"])
new_ids = authors_old.set_index('author')['author_id']
authors_old = (authors_old
    .set_index('author_id')
    ['author']
    .str.title())
display(authors_old.head())
display(new_ids.head())

author_id
restored256c86731f9118c3ad3a48ec               Alan Smithee
restorede74eaa2a6e8eb9de51813d70                Kim Shannon
restored59a454e2ce2e2b00e54b56ad    Kristin Sage Rockermann
restored769412f766e609eba47263b0                Kyle Reiter
restored052c5d4b676c00bff0f6f375              Kevin Adickes
Name: author, dtype: object

author
alan smithee               restored256c86731f9118c3ad3a48ec
kim shannon                restorede74eaa2a6e8eb9de51813d70
kristin sage rockermann    restored59a454e2ce2e2b00e54b56ad
kyle reiter                restored769412f766e609eba47263b0
kevin adickes              restored052c5d4b676c00bff0f6f375
Name: author_id, dtype: object

#### Put everything together


In [13]:
review_author_update = (original_authors
    .assign(author_id = lambda adf: adf
        .author_lower
        .map(new_ids.to_dict()))
    ['author_id'])
review_author_update

review_id
5929e2265e6ef959693250f6    restoredee3c3de4711302bbec49d5dd
5929e239d7a71d1ea569a421    restored38952617d98ef42ec86bcfee
5929e238c0084474cd0c4d38    restored59a454e2ce2e2b00e54b56ad
5929e23d5e6ef9596932510b    restored59a454e2ce2e2b00e54b56ad
5929e250d7a71d1ea569a432    restoredee3c3de4711302bbec49d5dd
                                          ...               
5929e40f5e6ef959693252cf    restoreda2412175615473c40a4b8753
5929e41113d197565213d769    restoredd54c8234de80e8041996ac74
5929e415eb335119a49ef1b2    restoredd54c8234de80e8041996ac74
5929e094d7a71d1ea569a266    restoredb8a3b4c1ed2658b1b8d12846
5929e0ef5e6ef95969324fb6    restoredb8a3b4c1ed2658b1b8d12846
Name: author_id, Length: 752, dtype: object


**authors** table modifications:
* For each *author_id* in *authors_old*:
  - Add the tuple (author_id, author, None).
  - The "None" place corresponds to the authors URL bio page (they have all been deleted from Pitchfork)

**review_authors** table modifications:
* For each *review_id* in *review_author_update*:  
  - Delete existing records associated with the given review_id
  - Add new records, association the newly (for old authors) created tuples (review_id, author_id)


In [ ]:
import sys
sys.path.append(str(Path.cwd().parent))

from scraper import types as t, db

In [ ]:
def get_connection_25(filepath=data):
    return sqlite3.connect(filepath / 'Pitchfork_Album_Reviews_2025_03_02.db')

In [ ]:
new_authors = [t.Author(author_id, author, None) for author_id, author in authors_old.items()]
db.insert_named_tuples(get_connection_25, new_authors, log=False)

In [ ]:
def batch_delete_review_authors(review_ids, con, batch_size=900):

    cur = con.cursor()
    for i in range(0, len(review_ids), batch_size):
        batch = review_ids[i : i + batch_size]
        sql_list = ",".join("?" * len(batch))
        query = f"DELETE FROM review_authors WHERE review_id IN ({sql_list})"
        cur.execute(query, batch)

    con.commit()


ids_to_clean = review_author_update.index.to_list()
batch_delete_review_authors(ids_to_clean, con25)


In [ ]:

review_authors = [t.Review_Authors(rid, aid) for rid, aid in review_author_update.items()]
db.insert_named_tuples(get_connection_25, review_authors, log=False)